In [3]:
# train_and_save_model.py

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
import os

# Set image size and paths
img_size = 224
train_dir = r"C:\Users\Ujwal M L\OneDrive\Documents\Internship\Hackathon\dataset\train"
val_dir = r"C:\Users\Ujwal M L\OneDrive\Documents\Internship\Hackathon\dataset\val"

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=20, zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir, target_size=(img_size, img_size), batch_size=32, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_dir, target_size=(img_size, img_size), batch_size=32, class_mode='categorical')

# Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Feature extraction

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

# Train
model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save the full model


Found 1586 images belonging to 2 classes.
Found 398 images belonging to 2 classes.
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_4 (Rescaling)        (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 normalization_2 (Normalization  (None, 224, 224, 3)  7          ['rescaling_4[0][0]']            
 )                                                                                                
         

In [24]:
model.save_weights("clean_weights.h5")
print(" Weights saved without attached metadata.")


 Weights saved without attached metadata.


In [3]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

# Set number of classes (update this to match your training)
NUM_CLASSES = 2  # or 3, based on your dataset

# Rebuild the architecture
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [4]:
model.load_weights("clean_weights.h5")
print(" Weights loaded successfully into the model.")


 Weights loaded successfully into the model.


In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model("plant_disease_model")
tflite_model = converter.convert()

# Save to file
with open("plant_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

print(" Converted to TFLite successfully.")


 Converted to TFLite successfully.
